In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
#Import CTD bottle nitrate data for calibration of SUNA

infile = './nitratewatersamples.txt'
NO3_bottle = pd.read_csv(infile, sep='\t', names = ["StationNr", "Depth", "Nitrate"])

AlignedFile = '%s.txt' % ('NO3_SUNA_Sample_aligned')
AlignedF = open(AlignedFile,'w')

Aligned = pd.DataFrame(columns=['Station','Depth','Nitrate','NitrateBinned'])

In [3]:
#Loop through SUNA files
for f_name in os.listdir('./SUNA_profiles/'):
    if f_name.endswith('.txt'):
        f_name_elements = f_name.split('sta')
        #print(f_name_elements)
        f_name_getnr = f_name_elements[1].split('.txt')
        infile = './SUNA_profiles/%s' % (f_name)
        SUNA_profile = pd.read_csv(infile, sep=' ')
        #From bin ctd code from Corday
        inc = 1 # define depth increment to bin by (m)
        depthlist = np.arange(0,np.round(np.max(SUNA_profile.Depth), decimals = 0) + inc, inc)
        df = pd.DataFrame(columns=['Station','Depth','NitrateBinned'])
        for j in depthlist:
            #Modify following to read in all profiles
            #loc = df[(df.Cast == i) & (np.round(df.Depth_m, decimals = 0) == j)]
            loc = SUNA_profile[(np.round(SUNA_profile.Depth, decimals = 0) == j)]
            Nitratebinned = np.mean((loc["Nitrate(umol/L)"]))
            df = df.append({'Station': int(f_name_getnr[0]),'Depth':j,'NitrateBinned':Nitratebinned}, ignore_index=True)
    for i in df.Depth:
        if pd.isna(df.NitrateBinned[df.Depth ==i].item()) == True:
            NitrateBinned_new_item = np.mean(df.NitrateBinned[(df.Depth == i+1)|(df.Depth == i-1)])
            df.NitrateBinned[(df.Depth == i)] = NitrateBinned_new_item
        else:
            NitrateBinned_new_item = df.NitrateBinned[df.Depth ==i].item()
            df.NitrateBinned[(df.Depth == i)] = NitrateBinned_new_item
    #Uncomment the following line if you want to overwrite the binned SUNA nitrate files
    df.to_csv('%s%s%s' % (f_name_elements[0],'binned_',f_name_elements[1]), header=None, index=None, sep='\t', mode='a')
    
    #Find corresponding depths between files
    for index, row in NO3_bottle.iterrows():
        for indexSUNA, rowSUNA in df.iterrows():
            if int(row['StationNr']) == int(rowSUNA['Station']) and int(row['Depth']) == int(rowSUNA['Depth']):
                Aligned = Aligned.append({'Station': (int(rowSUNA['Station'])),'Depth':row['Depth'],'Nitrate' : row['Nitrate'],'NitrateBinned':rowSUNA['NitrateBinned']}, ignore_index=True)
                #print(int(row['StationNr']),' ',rowSUNA['NitrateBinned'])

In [4]:
print(Aligned)

    Station  Depth  Nitrate  NitrateBinned
0      75.0    3.0   0.0861       3.720000
1      75.0   24.0   0.2979       9.726667
2      75.0   42.0   2.3519      14.600000
3      75.0  100.0  14.0838      17.393333
4      75.0  132.0  13.6796      17.445000
5     171.0    3.0   0.0653       8.580500
6     171.0   10.0  -0.0216      10.517500
7     171.0   23.0   1.0740      10.617500
8     171.0   35.0   4.0899       7.003125
9      20.0    1.0   0.0617            NaN
10     20.0   45.0   0.6030       5.970000
11     20.0  100.0  11.3285      17.790000
12     20.0  250.0  12.9157      16.170000
13     20.0  450.0  11.3803      16.222436
14    166.0    4.0   0.1210       4.917917
15    166.0   35.0   1.1324      12.470000
16    166.0   50.0   5.4596      13.500000
17    166.0  100.0  12.8708      18.747500
18    166.0  450.0  13.2613      18.143333
19    168.0    4.0   0.0195       5.763333
20    168.0   25.0   0.7417      13.045000
21    168.0   50.0   7.6599      17.016667
22    168.0

In [10]:
Aligned.to_csv('Aligned_BinnedSUNA_bottleSamples.csv', header=['Station','Depth','Nitrate Water Sample', 'Nitrate SUNA'], index=None, sep='\t', mode='a')


In [ ]:
#Skip below, just saved for reference

In [17]:
#Find corresponding depths between files
for index, row in NO3_bottle.iterrows():
    for indexSUNA, rowSUNA in df.iterrows():
        if int(row['StationNr']) == int(rowSUNA['Station']) and int(row['Depth']) == int(rowSUNA['Depth']):
            print(row['Depth'],' ', row['Nitrate'],' ', rowSUNA['NitrateBinned'])

1.0   0.2471   nan
30.0   0.5347   2.78
150.0   14.1829   nan
200.0   13.1632   15.48


In [ ]:
#Above depth of 150 is returning a nan for the binned nitrate

#Raw unbinned SUNA nitrate data around depth of 150:

#147.515478 15.020000 
#149.549457 NaN 
#151.236311 15.880000 